In [1]:
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!wget https://github.com/tesseract-ocr/tessdata_best/raw/main/tha.traineddata
!sudo mv tha.traineddata /usr/share/tesseract-ocr/4.00/tessdata/
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,889 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
import pandas as pd
import os

import pytesseract
from PIL import Image
import requests
from io import BytesIO

import nltk
from nltk.translate.bleu_score import sentence_bleu

In [3]:
!unzip -q /content/OCR.zip

<hr>

# OCR

In [31]:
def ocr_image(image_path):
    text = pytesseract.image_to_string(Image.open(image_path), lang='eng')
    # text = text.replace("\n","")
    # text = text.replace("\\x0"," ")
    # text = text.replace("\\x0c"," ")
    # text = text.replace("   "," ")
    text = text.replace("\n\x0c"," ")
    text = text.replace(" ",'')
    return text
ocr_image("/content/OCR/image/0000.jpg")

'12:00:00'

In [32]:
image_dir = "/content/OCR/image"
ocr_predictions = {}
# for i in range(0, len(os.listdir(image_dir))):
for i in range(0,500):
    img_id = f"{i:04}.jpg"
    image_path = os.path.join(image_dir, img_id)
    ocr_predictions[img_id] = ocr_image(image_path)

In [33]:
ocr_predictions

{'0000.jpg': '12:00:00',
 '0001.jpg': '12:00:01',
 '0002.jpg': '12:00:02',
 '0003.jpg': '12:00:03',
 '0004.jpg': '12:00:04',
 '0005.jpg': '12:00:05',
 '0006.jpg': '12:00:06',
 '0007.jpg': '12:00:07',
 '0008.jpg': '12:00:08',
 '0009.jpg': '12:00:09',
 '0010.jpg': '12:00:10',
 '0011.jpg': '42:00:11',
 '0012.jpg': '12:00:12',
 '0013.jpg': '12:00:13',
 '0014.jpg': '12:00:14',
 '0015.jpg': '12:00:15',
 '0016.jpg': '12:00:16',
 '0017.jpg': '12:00:17',
 '0018.jpg': '12:00:18',
 '0019.jpg': '12:00:19',
 '0020.jpg': '12:00:20',
 '0021.jpg': '12:00:21',
 '0022.jpg': '12:00:22',
 '0023.jpg': '12:00:23',
 '0024.jpg': '12:00:24',
 '0025.jpg': '42:00:25',
 '0026.jpg': '12:00:26',
 '0027.jpg': '12:00:27',
 '0028.jpg': '12:00:28',
 '0029.jpg': '12:00:29',
 '0030.jpg': '12:00:30',
 '0031.jpg': '12:00:31',
 '0032.jpg': '12:00:32',
 '0033.jpg': '12:00:33',
 '0034.jpg': '12:00:34',
 '0035.jpg': '12:00:35',
 '0036.jpg': '12:00:36',
 '0037.jpg': '12:00:37',
 '0038.jpg': '12:00:38',
 '0039.jpg': '12:00:39',


<hr>

# Bleu

In [34]:
ocr_data = pd.read_csv("/content/OCR/output.csv")

In [35]:
# ocr_predictions = {
#     '0001.jpg': '12:00:01',
#  '0002.jpg': '12:00:02',
#  '0003.jpg': '12:00:03',
#  '0004.jpg': '12:00:04',
#  '0005.jpg': '12:00:05',
#  '0006.jpg': '12:00:06',
#  '0007.jpg': '12:00:07',
#  '0008.jpg': '12:00:08',
#  '0009.jpg': '12:00:09',
#  '0010.jpg': '12:00:10',
#  '0011.jpg': '42:00:11',
#  '0012.jpg': '12:00:12',
#  '0013.jpg': '12:00:13',
#  '0014.jpg': '12:00:14',
#  '0015.jpg': '12:00:15',
#  '0016.jpg': '12:00:16'}

In [36]:
def bleu_df(ocr_data,ocr_predict):
    bleu_scores = []
    for _,row in ocr_data[:500].iterrows():
        img_id = row['img_id']                             #ดึง img id
        GT_text = row['img_time']                              #ดึง text
        Pd_text = ocr_predict.get(img_id, "")
        print(GT_text,Pd_text)

        # GT_tokens = [GT_text.split()]                      #ตัดคำ
        # Pd_tokens = Pd_text.split()                        #ตัดคำ
        # print(GT_tokens)

        # bleu_score = sentence_bleu(GT_text, Pd_text)
        if GT_text == Pd_text:
          bleu_score = 1
        else:
          bleu_score = 0
        bleu_scores.append((img_id, bleu_score))
        # print(bleu_scores)

        bleu_df = pd.DataFrame(bleu_scores, columns=['img_id', 'bleu_score'])
        bleu_df['bleu_score'] = bleu_df['bleu_score'].apply(lambda x: round(x,6))   # cast ค่า เอาแค่ 6 ตน มากกว่านี้เลยไม่สวย

    scores = [score for _, score in bleu_scores]
    average_bleu_score = sum(scores) / len(scores)
    return bleu_df,average_bleu_score


In [37]:
ocr_predictions

{'0000.jpg': '12:00:00',
 '0001.jpg': '12:00:01',
 '0002.jpg': '12:00:02',
 '0003.jpg': '12:00:03',
 '0004.jpg': '12:00:04',
 '0005.jpg': '12:00:05',
 '0006.jpg': '12:00:06',
 '0007.jpg': '12:00:07',
 '0008.jpg': '12:00:08',
 '0009.jpg': '12:00:09',
 '0010.jpg': '12:00:10',
 '0011.jpg': '42:00:11',
 '0012.jpg': '12:00:12',
 '0013.jpg': '12:00:13',
 '0014.jpg': '12:00:14',
 '0015.jpg': '12:00:15',
 '0016.jpg': '12:00:16',
 '0017.jpg': '12:00:17',
 '0018.jpg': '12:00:18',
 '0019.jpg': '12:00:19',
 '0020.jpg': '12:00:20',
 '0021.jpg': '12:00:21',
 '0022.jpg': '12:00:22',
 '0023.jpg': '12:00:23',
 '0024.jpg': '12:00:24',
 '0025.jpg': '42:00:25',
 '0026.jpg': '12:00:26',
 '0027.jpg': '12:00:27',
 '0028.jpg': '12:00:28',
 '0029.jpg': '12:00:29',
 '0030.jpg': '12:00:30',
 '0031.jpg': '12:00:31',
 '0032.jpg': '12:00:32',
 '0033.jpg': '12:00:33',
 '0034.jpg': '12:00:34',
 '0035.jpg': '12:00:35',
 '0036.jpg': '12:00:36',
 '0037.jpg': '12:00:37',
 '0038.jpg': '12:00:38',
 '0039.jpg': '12:00:39',


In [40]:
ocr_data[:10]

,img_id,img_time
0,0000.jpg,12:00:00
1,0001.jpg,12:00:01
2,0002.jpg,12:00:02
3,0003.jpg,12:00:03
4,0004.jpg,12:00:04
5,0005.jpg,12:00:05
6,0006.jpg,12:00:06
7,0007.jpg,12:00:07
8,0008.jpg,12:00:08
9,0009.jpg,12:00:09


In [41]:
df,lst = bleu_df(ocr_data,ocr_predictions)
print("Bleu now:",round(lst,5))
df

12:00:00 12:00:00
12:00:01 12:00:01
12:00:02 12:00:02
12:00:03 12:00:03
12:00:04 12:00:04
12:00:05 12:00:05
12:00:06 12:00:06
12:00:07 12:00:07
12:00:08 12:00:08
12:00:09 12:00:09
12:00:10 12:00:10
12:00:11 42:00:11
12:00:12 12:00:12
12:00:13 12:00:13
12:00:14 12:00:14
12:00:15 12:00:15
12:00:16 12:00:16
12:00:17 12:00:17
12:00:18 12:00:18
12:00:19 12:00:19
12:00:20 12:00:20
12:00:21 12:00:21
12:00:22 12:00:22
12:00:23 12:00:23
12:00:24 12:00:24
12:00:25 42:00:25
12:00:26 12:00:26
12:00:27 12:00:27
12:00:28 12:00:28
12:00:29 12:00:29
12:00:30 12:00:30
12:00:31 12:00:31
12:00:32 12:00:32
12:00:33 12:00:33
12:00:34 12:00:34
12:00:35 12:00:35
12:00:36 12:00:36
12:00:37 12:00:37
12:00:38 12:00:38
12:00:39 12:00:39
12:00:40 12:00:40
12:00:41 12:00:41
12:00:42 12:00:42
12:00:43 12:00:43
12:00:44 12:00:44
12:00:45 42:00:45
12:00:46 12:00:46
12:00:47 12:00:47
12:00:48 42:00:48
12:00:49 42:00:49
12:00:50 42:00:50
12:00:51 42:00:51
12:00:52 42:00:52
12:00:53 42:00:53
12:00:54 42:00:54
12:00:55 4

,img_id,bleu_score
0,0000.jpg,1
1,0001.jpg,1
2,0002.jpg,1
3,0003.jpg,1
4,0004.jpg,1
...,...,...
495,0495.jpg,0
496,0496.jpg,0
497,0497.jpg,0
498,0498.jpg,1
